In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
from numpy import genfromtxt


In [ ]:
xtrain = genfromtxt('https://storage.googleapis.com/tsaikevin-data/iris/iris_training.csv', delimiter=',')[1:,:4]
ytrain = genfromtxt('https://storage.googleapis.com/tsaikevin-data/iris/iris_training.csv', delimiter=',')[1:,4]
xtest = genfromtxt('https://storage.googleapis.com/tsaikevin-data/iris/iris_test.csv', delimiter=',')[1:,:4]
ytest = genfromtxt('https://storage.googleapis.com/tsaikevin-data/iris/iris_test.csv', delimiter=',')[1:,4]


In [ ]:
gpus = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']


m,n = xtrain.shape
c = np.unique(ytrain)
h = 6
len(c), n, h

In [ ]:
tf.reset_default_graph()

W = tf.Variable(tf.truncated_normal(shape=[n,h],stddev=0.01,dtype=tf.float32),name='W')
b = tf.Variable(tf.constant(0.0,shape=[h],dtype=tf.float32),name='b')
H = tf.Variable(tf.truncated_normal(shape=[h,len(c)],stddev=0.01,dtype=tf.float32),name='H')
B = tf.Variable(tf.constant(0.0,shape=[len(c)],dtype=tf.float32),name='B')
x = tf.placeholder(tf.float32,shape=[None,n],name='x')
y = tf.placeholder(tf.int64, [None],name='y')

def model_fn(x,y,W,b,H,B,lmbd,name='tower',infer=False):
    with tf.name_scope(name):
        logits = tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(x,W),b)),H),B)
        probs = tf.nn.softmax(logits,axis=1)
        ypred = tf.argmax(probs,dimension=1)
        if infer==False:
            reg = tf.multiply(lmbd,tf.reduce_sum(tf.square(W)) + tf.reduce_sum(tf.square(b)) + tf.reduce_sum(tf.square(H)) + tf.reduce_sum(tf.square(B)))
            loss = tf.losses.sparse_softmax_cross_entropy(y,logits) + reg
            dW,db,dH,dB = tf.gradients(xs=[W, b, H, B], ys=loss)
        else:
            loss = tf.zeros([1],dtype=tf.float32)
            dW = loss
            db = loss
            dH = loss
            dB = loss
        return ypred,loss,dW,db,dH,dB

init = tf.global_variables_initializer()

In [ ]:
l = np.arange(m)
batch_size = 16
steps = 10000

i = 0
lr = 0.005
lmbd = 0.01

ypred,loss,dW,db,dH,dB = model_fn(x,y,W,b,H,B,lmbd,name='tower0',infer=False)
y_,_,_,_,_,_           = model_fn(x,y,W,b,H,B,lmbd,name='tower0',infer=True)

dummy = [W.assign(W - lr*dW),b.assign(b - lr*db),H.assign(H - lr*dH),B.assign(B - lr*dB)]

sess = tf.Session()
sess.run(init)

for step in range(0,steps):
    j = l[(i + batch_size) % len(l)]
    if i>j:
        z = np.concatenate((np.arange(i,m),np.arange(j)))
    else:
        z = np.arange(i,j)
    i = l[j % len(l)]
    X = xtrain[z,:]
    Y = ytrain[z]

    preds,ls,_ = sess.run([ypred,loss,dummy],feed_dict={x:X,y:Y})
    
    if step%(steps/10)==0:
        print(ls)
    
Xt = xtest[:,:]
Yt = ytest[:]
y_ = sess.run([y_],feed_dict={x:Xt,y:Yt})
print('Test Accuracy:',np.sum(y_[0]==ytest),'/',len(ytest))


In [ ]:
gpus


In [ ]:
devs = len(gpus)
Ypred = []
Loss = []
DW = []
Db = []
DH = []
DB = []

for dev in range(0,devs):
    with tf.device(gpus[dev]):
        ypred,loss,dW,db,dH,dB = model_fn(x,y,W,b,H,B,lmbd,name='tower'+str(dev),infer=False)
        Ypred.append(ypred)
        Loss.append(loss)
        DW.append(dW)
        Db.append(db)
        DH.append(dH)
        DB.append(dB)
        

In [ ]:
print (tf.get_default_graph().as_graph_def())

In [ ]:
with tf.device('/cpu:0'):
    sYpred = tf.concat(Ypred,axis=0)
    sLoss = tf.add_n(Loss)
    sDW = tf.divide(tf.add_n(DW),devs)
    sDb = tf.divide(tf.add_n(Db),devs)
    sDH = tf.divide(tf.add_n(DH),devs)
    sDB = tf.divide(tf.add_n(DB),devs)
    
    dummy = [W.assign(W - lr*sDW),b.assign(b - lr*sDb),H.assign(H - lr*sDH),B.assign(B - lr*sDB)]